# In this notebook, I have selected 5 random course_ids and fetched the user_action data of these courses and saved them in an excel file.

In [ ]:
import pandas as pd
import numpy as np
import time, json, requests, yaml, random

with open("..\key.yaml", "r") as file:
    api_keys = yaml.safe_load(file)
# api_keys)

In [6]:
# Get all the category_IDs
df = pd.read_csv("all_category_ids.csv")
ids_for_demo = [22,25,27,28,29]
df_random = df[df["category_id"].isin(ids_for_demo)]
df_random

,category_id,name
4,22,English II
7,25,Programming in Python
9,27,Database Management Systems
10,28,Machine Learning Foundations
11,29,Modern Application Development I


In [7]:
def execute_query(query_id, query_params=None):
    DISCOURSE_BASE_URL = "https://discourse.onlinedegree.iitm.ac.in"
    GROUP_NAME = "discourse_analytics"
    API_KEY_GLOBAL= api_keys["API_KEY"]
    API_USERNAME = 'shubhamG'

    iteration_count = 0  # Initialize iteration counter
    results_list = []  # List to store results
    has_more_results = True  # Flag to control the loop for pagination

    # Check if query_params is provided
    if query_params is None:
        pass  # No parameters provided, continue with default
    else:
        # Ensure query_params is a dictionary
        if not isinstance(query_params, dict):
            raise ValueError("Query parameters must be a dictionary.")

    # Set up headers for the API request
    headers = {
        "Accept": "*/*",
        "Api-Key": API_KEY_GLOBAL,  # Get API key from userdata
        "Api-Username": API_USERNAME,  # Set the username for the API
        "Content-Type": "multipart/form-data"  # Set content type
    }

    # Loop until there are no more results
    while has_more_results:
        # Construct the request URL for the API
        request_url = f"{DISCOURSE_BASE_URL}/g/{GROUP_NAME}/reports/{query_id}/run"

        # Prepare the data payload for the request
        if query_params is not None:
            payload = {'page': str(iteration_count)}  # Add page number to payload
            payload.update(query_params)  # Update payload with additional query parameters
            data_payload = 'params=' + json.dumps(payload)  # Convert payload to JSON string
        else:
            data_payload = f'params={{"page": "{iteration_count}"}}'  # Default payload with page number

        try:
            # Send POST request to the API
            # print(data_payload)
            response = requests.request("POST", request_url, data=data_payload, headers=headers)
            response.raise_for_status()  # Raise an error for bad responses

            json_response = response.json()  # Parse the JSON response

            # Check if there are no results
            if json_response["result_count"] == 0:
                has_more_results = False  # No more results to fetch
                break

            # Iterate over the rows in the response
            for index in range(len(json_response['rows'])):
                # Append each row as a dictionary to the results list
                results_list.append(dict(zip(json_response['columns'], json_response['rows'][index])))

        except Exception as e:
            # Log key-related errors
            print(f'error: {e}')
            has_more_results = False  # Stop fetching results
            break

        iteration_count += 1  # Increment iteration count for pagination
        time.sleep(1.4)  # Wait before the next request

    results_dataframe = pd.DataFrame(results_list)  # Convert results list to DataFrame
    return results_dataframe  # Return the DataFrame with results

In [8]:
# Pick any random 5 values from category_ids
for row in df_random.itertuples():
    category_id = row.category_id
    category_name = row.name
    params = {"category_id":str(category_id),"start_date":"22/01/2024","end_date":'28/04/2024'}
    df = execute_query(103, query_params=params)
    df.to_excel(f"course_excel_data/{category_name}.xlsx", sheet_name="user_actions_data", index=False)